#### Importaciones

In [18]:
from db_class import DB
import pandas as pd

#### Instanciamos un objeto de la clase DB pasando como parámetro el nombre del fichero

In [19]:
base_datos = DB("ventas.db")

Connection is established to ventas.db


#### Cargamos en un dataframe los datos de la tabla de ventas utilizando la conexión de la base de datos

In [20]:
df_ventas = pd.read_sql_query("SELECT * from ventas", base_datos.conn)

#### Mostramos las primeras filas de la tabla

In [21]:
df_ventas.head()

,id,producto,cantidad,importe,ciudad,anyo,mes
0,1,1,100,None,Sevilla,2019,1
1,2,1,50,None,Madrid,2019,1
2,3,1,75,None,Sevilla,2019,2
3,4,1,25,None,Madrid,2019,3
4,5,2,40,None,Madrid,2019,2


#### Cargamos en un dataframe los datos de la tabla de productos utilizando la conexión de la base de datos

In [22]:
df_productos = pd.read_sql_query("SELECT * FROM productos", base_datos.conn)

In [23]:
df_productos.head()

,id,nombre,precio_unitario,coste_produccion
0,1,Barrita-NGA,5.50,3.2
1,2,Barrita-NGB,6.20,2.5
2,3,Barrita-NGC,7.50,4.5
3,4,Barrita-NGD,8.15,4.0


#### Hacemos un merge indicando las columnas que se igualan con left_on y right_on

In [24]:
df_mix = pd.merge(df_ventas,df_productos,how='inner',left_on='producto',right_on='id')

In [25]:
df_mix.head()

,id_x,producto,cantidad,importe,ciudad,anyo,mes,id_y,nombre,precio_unitario,coste_produccion
0,1,1,100,None,Sevilla,2019,1,1,Barrita-NGA,5.5,3.2
1,2,1,50,None,Madrid,2019,1,1,Barrita-NGA,5.5,3.2
2,3,1,75,None,Sevilla,2019,2,1,Barrita-NGA,5.5,3.2
3,4,1,25,None,Madrid,2019,3,1,Barrita-NGA,5.5,3.2
4,14,1,40,None,Madrid,2019,6,1,Barrita-NGA,5.5,3.2


#### Renombramos columnas para evitar confusiones

In [26]:
df_mix_renamed = df_mix.rename(columns={'id_x' : 'id_venta', 'id_y' : 'id_producto'})

In [27]:
df_mix_renamed.head()

,id_venta,producto,cantidad,importe,ciudad,anyo,mes,id_producto,nombre,precio_unitario,coste_produccion
0,1,1,100,None,Sevilla,2019,1,1,Barrita-NGA,5.5,3.2
1,2,1,50,None,Madrid,2019,1,1,Barrita-NGA,5.5,3.2
2,3,1,75,None,Sevilla,2019,2,1,Barrita-NGA,5.5,3.2
3,4,1,25,None,Madrid,2019,3,1,Barrita-NGA,5.5,3.2
4,14,1,40,None,Madrid,2019,6,1,Barrita-NGA,5.5,3.2


#### Agrupamos por producto y ciudad, después sumamos (se suman el resto de campos)

In [28]:
df_resultado = df_mix_renamed.groupby(by=["nombre","ciudad"]).sum(numeric_only=True)

In [29]:
df_resultado

id_venta  producto  cantidad  anyo  mes  id_producto  \
nombre      ciudad                                                          
Barrita-NGA Madrid         20         3       115  6057   10            3   
            Sevilla         4         2       175  4038    3            2   
Barrita-NGB Lugo           21         4       460  4038   11            4   
            Madrid         12         4       120  4038    6            4   
Barrita-NGC Lugo           10         3       200  2019    5            3   
            Madrid          9         3       140  2019    5            3   
            Sevilla         8         3        25  2019    4            3   
Barrita-NGD Madrid         24         8       275  4038   13            8   
            Sevilla        12         4       130  2019    7            4   

                     precio_unitario  coste_produccion  
nombre      ciudad                                      
Barrita-NGA Madrid             16.50               9.6  
            Sevilla            11.00               6.4  
Barrita-NGB Lugo               12.40               5.0  
            Madrid             12.40               5.0  
Barrita-NGC Lugo                7.50               4.5  
            Madrid              7.50               4.5  
            Sevilla             7.50               4.5  
Barrita-NGD Madrid             16.30               8.0  
            Sevilla             8.15               4.0

#### Filtramos para dejar las columnas que nos interesan: producto, ciudad y cantidad

In [30]:
df_resultado = df_mix_renamed[['nombre','ciudad','cantidad']].groupby(by=["nombre","ciudad"]).sum() 

In [31]:
df_resultado

cantidad
nombre      ciudad           
Barrita-NGA Madrid        115
            Sevilla       175
Barrita-NGB Lugo          460
            Madrid        120
Barrita-NGC Lugo          200
            Madrid        140
            Sevilla        25
Barrita-NGD Madrid        275
            Sevilla       130